In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from sklearn.metrics import precision_recall_curve 
from inspect import signature
from sklearn.metrics import average_precision_score

train=pd.read_csv('Train_Sample.csv',sep=';')

train_id=train.Id
train_num=train.drop(columns=train.columns[train.dtypes==object]).drop(columns='Id')
y=train.Status
train_num.drop(columns='Status',inplace=True)

from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=22)

for train_index, val_index in sss.split(train_num, y):
    print("TRAIN:", train_index, "VAL:", val_index)
    X_train, X_val = train_num.iloc[train_index,:], train_num.iloc[val_index,:]
    y_train, y_val = y[train_index], y[val_index]

TRAIN: [ 4935 25551 61271 ...  9337 12070 29357] VAL: [14537 59552 25318 ... 38813   750 20012]


In [2]:
pd.set_option('display.max_columns',250)
X_train.describe()

,par1,par2,par3,par4,par5,par6,par7,par8,par9,par10,par11,par12,par13,par14,par15,par16,par17,par18,par19,par20,par21,par22,par23,par24,par25,par26,par27,par28,par29,par30,par31,par32,par33,par34,par35,par36,par37,par38,par39,par40,par41,par42,par43,par44,par45,par46,par47,par48,par49,par50,par51,par52,par54,par55,par56,par57,par59,par63,par64,par65,par66,par67,par69,par70,par72,par73,par74,par75,par76,par77,par78,par79,par80,par81,par82,par83,par84,par85,par86,par87,par88,par89,par90,par91,par92,par93,par94,par95,par96,par97,par98,par99,par100,par101,par102,par103,par104,par105,par106,par107,par108,par109,par110,par111,par112,par113,par114,par115,par116,par117,par118,par119,par120,par121,par122,par123,par124,par125,par126,par127,par128,par129,par130,par131,par132,par133,par134,par135,par136,par137,par138,par139,par140,par141,par142,par143,par144,par145,par146,par147,par148,par149,par150,par151,par152,par153,par154,par155,par156,par157,par158,par159,par160,par161,par162,par163,par164,par165,par166,par167,par168,par169,par170,par171,par172,par173,par174,par175,par176,par177,par178,par179,par180,par181,par182,par183,par184,par185,par186,par187,par188,par189,par190,par191,par192,par193,par194,par195,par196,par197,par198,par199,par201,par202,par203,par204,par205,par206,par207,par208,par209,par210,par211,par212,par213,par214,par215,par216,par217,par218,par219,par220,par221,par222,par223
count,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,4.948000e+04,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,4.948000e+04,4.948000e+04,4.948000e+04,4.948000e+04,49480.000000,49480.000000,49480.000000,49480.000000,4.948000e+04,49480.000000,4.948000e+04,4.948000e+04,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,4.948000e+04,4.948000e+04,4.948000e+04,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,4.948000e+04,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.00000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,49480.000000,4.9

### KNN

In [3]:
from pyod.models.knn import KNN

In [62]:
oda_name = 'KNN'
oda = KNN(contamination=.1, n_jobs=-1)
oda.fit(X_train)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
  radius=1.0)

In [63]:
%%time
y_out=oda.predict(X_train)

Wall time: 11min 13s


In [64]:
y_out=pd.Series(y_out, index=X_train.index, name='flag')

In [65]:
pd.concat([X_train, y_out],axis=1)

,par1,par2,par3,par4,par5,par6,par7,par8,par9,par10,par11,par12,par13,par14,par15,par16,par17,par18,par19,par20,par21,par22,par23,par24,par25,par26,par27,par28,par29,par30,par31,par32,par33,par34,par35,par36,par37,par38,par39,par40,par41,par42,par43,par44,par45,par46,par47,par48,par49,par50,par51,par52,par54,par55,par56,par57,par59,par60,par63,par64,par65,par66,par67,par68,par69,par70,par72,par73,par74,par75,par76,par77,par78,par79,par80,par81,par82,par83,par84,par85,par86,par87,par88,par89,par90,par91,par92,par93,par94,par95,par96,par97,par98,par99,par100,par101,par102,par103,par104,par105,par106,par107,par108,par109,par110,par111,par112,par113,par114,par115,par116,par117,par118,par119,par120,par121,par122,par123,par124,par125,par126,par127,par128,par129,par130,par131,par132,par133,par134,par135,par136,par137,par138,par139,par140,par141,par142,par143,par144,par145,par146,par147,par148,par149,par150,par151,par152,par153,par154,par155,par156,par157,par158,par159,par160,par161,par162,par163,par164,par165,par166,par167,par168,par169,par170,par171,par172,par173,par174,par175,par176,par177,par178,par179,par180,par181,par182,par183,par184,par185,par186,par187,par188,par189,par190,par191,par192,par193,par194,par195,par196,par197,par198,par199,par200,par201,par202,par203,par204,par205,par206,par207,par208,par209,par210,par211,par212,par213,par214,par215,par216,par217,par218,par219,par220,par221,par222,par223,flag
4935,11,0.000000,0,0,1828,13538.400000,1.000000,39123,3.571429,2.666667,0.000000,0,0,12,100,0,0,71153,4010,19569.900000,195699,35,0.000000,0.000000,0.000000,0,14,2924,174911,44,1,0.500000,4,0.000000,0,0,2,22,2.153846,57,43878,18917,0,0.031462,0.997407,2.000000,0.000000,4314.00,30,19,1,25,4,8077,4,5,63,True,8,4,254,6000,0,False,-23,3,1.138667,2276,1,3330,1,3330,1.110000,1500,2276,1500,3330,0.888333,1.11,2665,1,1500,1500,0.500000,0.000000,1500,1500,0.500000,0,0,0.000000,0,0.000000,0,0.000000,1500,0.000000,0,0.000000,1500,1,1500,0,0,0,7,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0.000000,0,0,0.0,0.0000,0.0000,0,0.0,0.0,0.0000,0,1,1.0000,0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0,1,1,1708.0,1708.0,1.1387,0.0000,0.0,1.0000,1.1387,0.0000,1,1708.0,1.0000,1.1387,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.0000,1.0000,1,1708.0,1.0000,0.0000,1.0000,1708.0000,1.0000,0.000000,0.333333,2.333333,0,0.000000,0.000000,4314.00,4,0.000000,True,1.000000,1.000000,0.000000,28,25,3000,3150.000000,44100,0,0,0,3000,3150.000000,0,0.000000,0,0,0,0.000000,0,3131,55146.615380,19,0
25551,18,0.214286,3,0,478,8488.000000,1.000000,58802,2.230769,2.800000,0.000000,0,2,17,100,3870,3,7015,534,2477.714286,34688,17,1872.777778,0.071429,0.071429,33710,0,458,617835,17,5,1.083333,13,0.000000,0,1,0,10,0.000000,7,17346,12137,0,0.298656,0.999306,2.500000,0.733333,5750.82,30,16,1,52,11,8641,10,3,99,True,13,3,210,10000,0,True,-16,5,1.256746,2520,0,0,1,1650,0.426357,1650,1650,1650,0,0.000000,0.00,0,3,3410,2520,0.651163,0.426357,1650,1650,0.426357,2500,2500,0.645995,2500,0.645995,2500,0.645995,1650,0.000000,1650,0.426357,1650,3,2520,2,1,0,3,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0.000000,0,1,1797.0,0.3333,599.0000,1,1797.0,1797.0,0.0000,0,2,0.6667,0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,345.0,5,3,160.0,4904.0,1.2567,0.7304,0.0,0.0000,0.0000,0.0000,1,1797.0,0.7304,1.0891,0.0,0.0,0.0000,0.0000,4904.0,4904.0,4904.0,4904.0,4904.0,0.9980,1.4381,0.0000,160.0,160.0,2800.0,160.0,2295.0,1.0526,1.1200,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.3492,0.6667,5,160.0,0.5047,0.6667,1.6667,3167.0000,0.9469,0.000000,1.000000,1.000000,3,0.000000,1.500000,5750.82,5,0.000000,False,1.000000,0.646566,0.545699,29,52,0,0.000000,0,0,0,0,1000,3045.454545,10,22.000000,5,0,0,0.000000,0,786,8534.437500,26,0
61271,13,0.200000,2,0,0,14813.833330,1.666667,0,0.000000,4.235294,0.588235,10,0,0,100,0,0,60818,1217,11628.000000,127908,69,0.000000,0.090909,0.000000,0,2,1439,205834,17,2,0.750000,6,0.000000,0,

In [66]:
from collections import Counter

In [67]:
Counter(y_out)

Counter({0: 40557, 1: 8923})

In [68]:
## Choosing only rows that are inliers 
X_fil1, y_fil1=X_train[y_out==0], y_train[y_out==0]

In [69]:
clf = LogisticRegression(random_state=22, solver='lbfgs',class_weight='balanced',n_jobs=-1).fit(X_fil1, y_fil1)

roc_auc_score(y_val, clf.predict_proba(X_val)[:,1] )

0.7798200450577879

In [70]:
y_pred=clf.predict_proba(X_val)[:,1]
ave_prec=average_precision_score(y_val,y_pred)  
print(ave_prec)

0.20793314300934565


### Eliiptic Envelope

In [71]:
from sklearn.covariance import EllipticEnvelope

In [85]:
%%time
cov = EllipticEnvelope(random_state=22, support_fraction=1)
cov.fit(X_train)
y_ee=cov.predict(X_train)

print( Counter( y_ee ) )

## Choosing only rows that are inliers 
X_fil2, y_fil2=X_train[y_ee==1], y_train[y_ee==1]

clf2 = LogisticRegression(random_state=22, solver='lbfgs',class_weight='balanced',n_jobs=-1).fit(X_fil2, y_fil2)

print( roc_auc_score(y_val, clf2.predict_proba(X_val)[:,1] ) )

y_pred=clf2.predict_proba(X_val)[:,1]
ave_prec=average_precision_score(y_val,y_pred)  
print(ave_prec)

Counter({1: 44532, -1: 4948})
0.7894995173016293
0.21469359743684968
Wall time: 3min 5s


### Isolation Forest

In [86]:
from sklearn.ensemble import IsolationForest

In [89]:
%%time
isf=IsolationForest(behaviour='new',contamination=.1 ,random_state=22)
isf.fit(X_train)
y_isf=isf.predict(X_train)

print( Counter( y_isf ) )

## Choosing only rows that are inliers 
X_fil3, y_fil3=X_train[y_isf==1], y_train[y_isf==1]

clf3 = LogisticRegression(random_state=22, solver='lbfgs',class_weight='balanced',n_jobs=-1).fit(X_fil3, y_fil3)

print( roc_auc_score(y_val, clf3.predict_proba(X_val)[:,1] ) )

y_pred=clf3.predict_proba(X_val)[:,1]
ave_prec=average_precision_score(y_val,y_pred)  
print(ave_prec)

Counter({1: 44532, -1: 4948})
0.7617919180210213
0.18799401745985697
Wall time: 1min 7s
